In [1]:
import numpy as np
import sys
sys.path.append('./')
import auxiliary_bed_functions as abf
chr_list = abf.getChrList()

RNAseq_input = '../tensor_input/RNAseq_input/'
output_dir = '../'

In [2]:
# Read the canonical gene table for the hg19 genome.
# The table was downloaded from the UCSC table browser.
# Select the knownCanonical table and select the
# three genomic coordinate fields, known gene ID
# and geneSymbol fields. The canonical gene regions
# are used for defining a gene regulatory region to 
# compare to histone modification data
gene_file = open('./knownCanonical_hg19.txt','r')

gene_table = []
for gene in gene_file:
    if(gene[0] != '#'):
        if(gene.strip().split('\t')[0] not in chr_list):
            continue
        gene_table.append(gene.strip().split('\t'))
        
gene_file.close()


# The isoform table can be obtained from the UCSC Table Browser.
# Select the knownGene table for the hg19 genome and select the
# first 5 fields for output.
isoform_file = open('./knownGene_hg19.txt','r')

isoform_table = []
for iso in isoform_file:
    if(iso[0] != '#'):
        isoform_table.append(iso.strip().split('\t'))
    
isoform_file.close()

# Get a strand for each gene in the canonical
# gene table. If the official protein symbol from the
# canonical gene table has multiple isoforms in the 
# isoform table, the strand of the first isoform is used.
# The strand is used to define the TSS as 5' coordinate 
# on the respected strand.
for i in range(0,len(gene_table)):
    for iso in isoform_table:
        if(gene_table[i][3] == iso[0]):
            gene_table[i].append(iso[2])
            break

In [3]:
# Get the TSS for each canonical gene and sort these by
# chromosome.

TSS_sorted_genes = [[] for c in chr_list]
TSS_loc = [[] for c in chr_list]

for i in range(0,len(gene_table)):
    gene_table[i][1] = int(gene_table[i][1])
    gene_table[i][2] = int(gene_table[i][2])

gene_table_byChr = [[] for i in range(0,len(chr_list))]
for i in range(len(gene_table)):
    gene_table_byChr[chr_list[gene_table[i][0]]].append(gene_table[i]) 
            
for c in range(len(chr_list)):
    for i in range(len(gene_table_byChr[c])):
        if(gene_table_byChr[c][i][5] == '+'):
            TSS_loc[c].append(gene_table_byChr[c][i][1])
        elif(gene_table_byChr[c][i][5] == '-'):
            TSS_loc[c].append(gene_table_byChr[c][i][2])
        else:
            print('No strand specified', c, i)       
            
for c in range(len(chr_list)):
    TSS_sort = np.sort(TSS_loc[c])
    TSS_index = np.argsort(TSS_loc[c])
    for i in range(len(TSS_sort)):
        TSS_sorted_genes[c].append([gene_table_byChr[c][TSS_index[i]][0],
                                    TSS_sort[i], gene_table_byChr[c][TSS_index[i]][3],
                                    gene_table_byChr[c][TSS_index[i]][4]])

In [4]:
# For each TSS get the region extending 10kb in the 5' and 3'
# direction of the TSS.
reg_regions = open('../gene_TSS_regulatory_regions10kb.tsv','w+')
for c in range(len(chr_list)):
    for i in range(len(TSS_sorted_genes[c])):
        reg_regions.write("{}\t{}\t{}\t{}\t{}\n".format(TSS_sorted_genes[c][i][0],
                                                               max(int(TSS_sorted_genes[c][i][1])-10000,0),
                                                               int(TSS_sorted_genes[c][i][1])+10000,
                                                               TSS_sorted_genes[c][i][2],
                                                               TSS_sorted_genes[c][i][3]))
reg_regions.close()

In [5]:
## Filter genes by FPKM

In [6]:
# The hg19 knownGene gtf annotation file. This
# is used to obtain the gene length for FPKM
# calculations.
gtf_file_name = './genes_hg19.gtf'

# Feature counts for each of the samples
file_list_L = ['L_pt1_RNAseq_alignReadsPerGene.tsv',
               'L_pt2_RNAseq_alignReadsPerGene.tsv',
               'L_pt3_RNAseq_alignReadsPerGene.tsv',
               'L_pt4_RNAseq_alignReadsPerGene.tsv',
               'L_pt5_RNAseq_alignReadsPerGene.tsv',
               'L_pt6_RNAseq_alignReadsPerGene.tsv',
               'L_pt7_RNAseq_alignReadsPerGene.tsv',
               'L_pt8_RNAseq_alignReadsPerGene.tsv',
               'L_pt9_RNAseq_alignReadsPerGene.tsv',
               'L_pt10_1_RNAseq_alignReadsPerGene.tsv',
               'L_pt11_RNAseq_alignReadsPerGene.tsv',
               'L_pt12_RNAseq_alignReadsPerGene.tsv',
               'L_pt13_RNAseq_alignReadsPerGene.tsv',
               'L_pt14_RNAseq_alignReadsPerGene.tsv',
               'L_pt15_RNAseq_alignReadsPerGene.tsv',
               'L_pt17_RNAseq_alignReadsPerGene.tsv',
               'L_pt18_RNAseq_alignReadsPerGene.tsv',
               'L_pt19_RNAseq_alignReadsPerGene.tsv',
               'L_pt20_RNAseq_alignReadsPerGene.tsv',
               'L_pt21_RNAseq_alignReadsPerGene.tsv']

file_list_M = ['M_pt1_RNAseq_alignReadsPerGene.tsv',
               'M_pt2_RNAseq_alignReadsPerGene.tsv',
               'M_pt3_RNAseq_alignReadsPerGene.tsv',
               'M_pt4_RNAseq_alignReadsPerGene.tsv',
               'M_pt5_RNAseq_alignReadsPerGene.tsv',
               'M_pt6_RNAseq_alignReadsPerGene.tsv',
               'M_pt7_RNAseq_alignReadsPerGene.tsv',
               'M_pt8_RNAseq_alignReadsPerGene.tsv',
               'M_pt9_RNAseq_alignReadsPerGene.tsv',
               'M_pt10_RNAseq_alignReadsPerGene.tsv',
               'M_pt11_RNAseq_alignReadsPerGene.tsv',
               'M_pt12_RNAseq_alignReadsPerGene.tsv',
               'M_pt13_RNAseq_alignReadsPerGene.tsv',
               'M_pt14_RNAseq_alignReadsPerGene.tsv',
               'M_pt15_RNAseq_alignReadsPerGene.tsv',
               'M_pt17_RNAseq_alignReadsPerGene.tsv',
               'M_pt18_RNAseq_alignReadsPerGene.tsv',
               'M_pt19_RNAseq_alignReadsPerGene.tsv',
               'M_pt20_RNAseq_alignReadsPerGene.tsv',
               'M_pt21_RNAseq_alignReadsPerGene.tsv']


In [7]:
def lengthOfUnion(union):
    # Get the length of the union of exons on each 
    # chromosome. Return the minimal length if exons
    # are found on multiple chromosomes for a given
    # gene_id.
    length = [0 for c in range(len(chr_list))]
    for c, chrom in enumerate(chr_list):
        for i in range(len(union[c])):
            length[c] += union[c][i][2]-union[c][i][1] + 1
    return min(np.array(length)[np.where(np.array(length) > 0)[0]])

def lengthOfExons(region_list):
    # Take the union of the exons on each chromosome.
    region_chr_list = [[] for c in range(len(chr_list))]
    for region in region_list:
        region_chr_list[chr_list[region[0]]].append(region)
    get_union = abf.unionOfRegions(region_chr_list, 1)
    return lengthOfUnion(get_union)

def readGTF(gtf_file_name):
    # Read the gtf file and get the total exon length
    # for each gene_id.
    gtf_file = open(gtf_file_name, 'r')
    gene_list = []
    gene_dic = {}
    n = 0
    region_list = []
    for line in gtf_file:
        l = line.strip().split('\t')
        if(l[2] != 'exon'):
            continue
        if(l[0] not in chr_list):
            continue
        # Extract the gene_id
        gene_name = l[-1].split(';')[0]
        gene_name = gene_name.split(' ')[1]
        gene_name = gene_name.strip('"') 
        gene_list.append(gene_name)
        # Get the list of all exons for each gene_id.
        if(gene_name not in gene_dic):
            gene_dic[gene_name] = [[l[0],int(l[3]),int(l[4])]]
        else:
            gene_dic[gene_name].append([l[0],int(l[3]),int(l[4])])
    # Get the length of the union of all exons.
    gene_length_list = {}
    for gene in gene_dic:
        length = lengthOfExons(gene_dic[gene])
        gene_length_list[gene] = length
    return gene_length_list

def getFPKM(gene_count_list, gene_length):
    # Given the file of gene counts (two coloumns, one 
    # for gene name and one for the number of reads aligning
    # to this gene) and a dictionary of gene lengths, compute
    # the FPKM for each gene.
    gene_FPKM = [[] for i in range(len(gene_count_list))]
    total = sum([gene_count_list[i][1] for i in range(len(gene_count_list))])
    for i in range(len(gene_count_list)):
        FPKM = (float(gene_count_list[i][1]) * 1e9)/(float(gene_length[gene_count_list[i][0]])*total)
        gene_FPKM[i] = [gene_count_list[i][0], FPKM]

    return gene_FPKM

def readGeneCount(gene_count_name):
    # Read the gene count files.
    gene_count_file = open(gene_count_name)
    gene_count_list = []
    for line in gene_count_file:
        l = line.strip().split('\t')
        gene_count_list.append([l[0],int(l[1])])    
    return gene_count_list

In [8]:
gene_length = readGTF(gtf_file_name)

In [9]:
FPKM_matrix = np.zeros((26334, 2*len(file_list_L)))
for i in range(len(file_list_M)):
    FPKM_list = getFPKM(readGeneCount(RNAseq_input + file_list_M[i]),gene_length)
    for j in range(len(FPKM_list)):
        FPKM_matrix[j][i] = FPKM_list[j][1]
        
for i in range(len(file_list_L)):
    FPKM_list = getFPKM(readGeneCount(RNAseq_input + file_list_L[i]),gene_length)
    for j in range(len(FPKM_list)):
        FPKM_matrix[j][i+len(file_list_M)] = FPKM_list[j][1]
        


In [10]:
read_counts = readGeneCount(RNAseq_input + file_list_L[0])
gene_names = [read_counts[i][0] for i in range(len(read_counts))]


In [11]:
# Select only genes with at least 1 FPKM in at
# least one of the samples.
FPKM_gene_list = []
for i in range(len(FPKM_matrix)):
    m = 0
    for j in range(len(FPKM_matrix[i])):
        if(FPKM_matrix[i][j] >= 1.0):
            m+=1
    if(m >= 1):
        FPKM_gene_list.append(gene_names[i])

# Write the list of expressed genes for later use
output = open(output_dir + 'expressedGenes_final_any.txt','w+')
for i in range(len(FPKM_gene_list)):
    # Remove genes with length less than 200
    if(gene_length[FPKM_gene_list[i]] <= 200):
        continue
    output.write("{}\n".format(FPKM_gene_list[i]))
output.close()